# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

In [4]:
t.a

tensor([[ 0.2440, -0.5656, -2.1488],
        [ 1.1303, -0.3813,  0.1548]])

In [5]:
%timeit t.a

83.2 ns ± 0.981 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

In [7]:
%timeit t.a = new_value

86.6 ns ± 1.66 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.2440, -0.5656, -2.1488],
               [ 1.1303, -0.3813,  0.1548]]),
    x: Batch(
           c: tensor([[ 1.7448, -0.8182,  0.1793, -1.6214],
                      [ 0.4989, -2.4102, -0.6332,  0.7780],
                      [ 0.1867, -0.3475, -0.3406, -0.3171]]),
       ),
)

In [10]:
b.a

tensor([[ 0.2440, -0.5656, -2.1488],
        [ 1.1303, -0.3813,  0.1548]])

In [11]:
%timeit b.a

79.1 ns ± 1.14 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.5999,  1.3027, -1.1038],
               [-2.4148,  0.6310,  0.0567]]),
    x: Batch(
           c: tensor([[ 1.7448, -0.8182,  0.1793, -1.6214],
                      [ 0.4989, -2.4102, -0.6332,  0.7780],
                      [ 0.1867, -0.3475, -0.3406, -0.3171]]),
       ),
)

In [13]:
%timeit b.a = new_value

641 ns ± 5.36 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.04 µs ± 5.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.4 µs ± 199 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

166 µs ± 1.91 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

163 µs ± 2.2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

In [21]:
%timeit t_stack(trees)

37.9 µs ± 447 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

In [23]:
%timeit t_cat(trees)

35.9 µs ± 123 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

75 µs ± 608 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.2440, -0.5656, -2.1488],
                [ 1.1303, -0.3813,  0.1548]],
       
               [[ 0.2440, -0.5656, -2.1488],
                [ 1.1303, -0.3813,  0.1548]],
       
               [[ 0.2440, -0.5656, -2.1488],
                [ 1.1303, -0.3813,  0.1548]],
       
               [[ 0.2440, -0.5656, -2.1488],
                [ 1.1303, -0.3813,  0.1548]],
       
               [[ 0.2440, -0.5656, -2.1488],
                [ 1.1303, -0.3813,  0.1548]],
       
               [[ 0.2440, -0.5656, -2.1488],
                [ 1.1303, -0.3813,  0.1548]],
       
               [[ 0.2440, -0.5656, -2.1488],
                [ 1.1303, -0.3813,  0.1548]],
       
               [[ 0.2440, -0.5656, -2.1488],
                [ 1.1303, -0.3813,  0.1548]]]),
    x: Batch(
           c: tensor([[[ 1.7448, -0.8182,  0.1793, -1.6214],
                       [ 0.4989, -2.4102, -0.6332,  0.7780],
                       [ 0.1867, -0.3475, -0.3406, -0.3171]],
         

In [26]:
%timeit Batch.stack(batches)

92.5 µs ± 785 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.2440, -0.5656, -2.1488],
               [ 1.1303, -0.3813,  0.1548],
               [ 0.2440, -0.5656, -2.1488],
               [ 1.1303, -0.3813,  0.1548],
               [ 0.2440, -0.5656, -2.1488],
               [ 1.1303, -0.3813,  0.1548],
               [ 0.2440, -0.5656, -2.1488],
               [ 1.1303, -0.3813,  0.1548],
               [ 0.2440, -0.5656, -2.1488],
               [ 1.1303, -0.3813,  0.1548],
               [ 0.2440, -0.5656, -2.1488],
               [ 1.1303, -0.3813,  0.1548],
               [ 0.2440, -0.5656, -2.1488],
               [ 1.1303, -0.3813,  0.1548],
               [ 0.2440, -0.5656, -2.1488],
               [ 1.1303, -0.3813,  0.1548]]),
    x: Batch(
           c: tensor([[ 1.7448, -0.8182,  0.1793, -1.6214],
                      [ 0.4989, -2.4102, -0.6332,  0.7780],
                      [ 0.1867, -0.3475, -0.3406, -0.3171],
                      [ 1.7448, -0.8182,  0.1793, -1.6214],
                      [ 0.4989, -

In [28]:
%timeit Batch.cat(batches)

174 µs ± 1.22 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

398 µs ± 6.39 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
